In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sub = pd.read_csv('submit.csv')

In [3]:
train.tail()

,id,title,author,text,label
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1
20799,20799,What Keeps the F-35 Alive,David Swanson,"David Swanson is an author, activist, journa...",1


In [4]:
train.shape

(20800, 5)

In [5]:
train['label'].value_counts()

label
1    10413
0    10387
Name: count, dtype: int64

In [6]:
train.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [7]:
train.dropna(inplace=True)
test.dropna(inplace=True)

In [8]:
train.shape

(18285, 5)

In [9]:
X = train.iloc[:, :-1]
y = train.iloc[:, -1]

In [10]:
X.shape

(18285, 4)

In [11]:
y.shape

(18285,)

In [12]:
import tensorflow as tf
tf.__version__

'2.16.1'

In [13]:
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential

In [14]:
### Vocabulary size
voc_size=5000

__________

## One Hot representation

_____

In [15]:
messages = X.copy()

In [16]:
messages['title'][:10]

0     House Dem Aide: We Didn’t Even See Comey’s Let...
1     FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                     Why the Truth Might Get You Fired
3     15 Civilians Killed In Single US Airstrike Hav...
4     Iranian woman jailed for fictional unpublished...
5     Jackie Mason: Hollywood Would Love Trump if He...
7     Benoît Hamon Wins French Socialist Party’s Pre...
9     A Back-Channel Plan for Ukraine and Russia, Co...
10    Obama’s Organizing for Action Partners with So...
11    BBC Comedy Sketch "Real Housewives of ISIS" Ca...
Name: title, dtype: object

In [17]:
messages.reset_index(inplace=True)
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [18]:
import nltk
import re
from nltk.corpus import stopwords

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/klsbelousov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [21]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

1. Dataset
2. Independent n Dependent features
3. Cleaning the data a) Stemming 2) Stopwords 

In [22]:
onehot_repr = [one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[1957, 571, 2108, 1813, 1312, 4056, 3886, 3149, 2031, 4209],
 [3977, 1657, 4890, 1441, 2509, 38, 413],
 [906, 3506, 4242, 862],
 [1745, 605, 1288, 2752, 1959, 4005],
 [1358, 2509, 392, 1264, 1935, 2976, 2509, 647, 3511, 2595],
 [2894,
  4750,
  3265,
  1025,
  1593,
  2496,
  4628,
  4010,
  2819,
  3261,
  940,
  2092,
  867,
  436,
  413],
 [3172, 462, 2586, 3743, 2853, 4955, 2341, 1813, 2940, 214, 1858],
 [4147, 1317, 3889, 1695, 3071, 1415, 2496, 666, 2940, 214, 1858],
 [2730, 3895, 1324, 483, 2444, 434, 3556, 2817, 2496, 3215],
 [841, 4173, 542, 1650, 1907, 2965, 1262, 2194],
 [1775, 2909, 1881, 4312, 3570, 1049, 4502, 3952, 240, 4162, 3115],
 [2752, 1649, 1312, 434, 2496, 3071],
 [4335, 454, 3424, 5, 886, 2609, 2255, 3810, 1864],
 [4454, 936, 2782, 2762, 396, 2455, 1020, 2940, 214, 1858],
 [4570, 1120, 2567, 256, 1527, 2940, 214, 1858],
 [1514, 3032, 747, 914, 332, 3708, 2736, 210, 685, 1664],
 [220, 1706, 1657],
 [329, 3648, 697, 2947, 2496, 2701, 927, 413],
 [1711, 4518, 4890,

______

## Embedding Representation

______

In [23]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[1957  571 2108 ...    0    0    0]
 [3977 1657 4890 ...    0    0    0]
 [ 906 3506 4242 ...    0    0    0]
 ...
 [2360 3787  139 ...    0    0    0]
 [ 714 3071 4381 ...    0    0    0]
 [ 737   96 2140 ...    0    0    0]]


In [25]:
## Creating model
embedding_vector_features=40 ##features representation

model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_shape=(sent_length, )))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

print(model.summary())

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:81: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 20, 40)         │       200,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 100)            │        56,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 256,501 (1001.96 KB)

 Trainable params: 256,501 (1001.96 KB)

 Non-trainable params: 0 (0.00 B)

None


In [26]:
len(embedded_docs),y.shape

(18285, (18285,))

In [27]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [28]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.2, random_state=42)

In [30]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.7836 - loss: 0.3960 - val_accuracy: 0.9166 - val_loss: 0.2036
Epoch 2/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9517 - loss: 0.1303 - val_accuracy: 0.9191 - val_loss: 0.2221
Epoch 3/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9692 - loss: 0.0899 - val_accuracy: 0.9169 - val_loss: 0.2119
Epoch 4/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9777 - loss: 0.0689 - val_accuracy: 0.9076 - val_loss: 0.2857
Epoch 5/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9843 - loss: 0.0507 - val_accuracy: 0.9051 - val_loss: 0.3424
Epoch 6/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9863 - loss: 0.0415 - val_accuracy: 0.9021 - val_loss: 0.3322
Epoch 7/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9886 - loss: 0.0354 - val_accuracy: 0.9078 - val_loss: 0.3921
Epoch 8/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9919 - loss: 0.0278 - val_accu

In [42]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_shape=(sent_length, )))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:81: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [43]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.7605 - loss: 0.4166 - val_accuracy: 0.9171 - val_loss: 0.1992
Epoch 2/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9482 - loss: 0.1386 - val_accuracy: 0.9155 - val_loss: 0.2033
Epoch 3/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9646 - loss: 0.0969 - val_accuracy: 0.9182 - val_loss: 0.2043
Epoch 4/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9738 - loss: 0.0820 - val_accuracy: 0.9111 - val_loss: 0.2730
Epoch 5/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9751 - loss: 0.0728 - val_accuracy: 0.9087 - val_loss: 0.2349
Epoch 6/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9798 - loss: 0.0626 - val_accuracy: 0.9027 - val_loss: 0.2911
Epoch 7/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9847 - loss: 0.0481 - val_accuracy: 0.9024 - val_loss: 0.3915
Epoch 8/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9857 - loss: 0.0407 - val_accu

In [44]:
y_pred = model.predict(X_test)

115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [45]:
y_pred = np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [46]:
from sklearn.metrics import confusion_matrix

In [47]:
confusion_matrix(y_test,y_pred)

array([[1883,  199],
       [ 147, 1428]])

In [48]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.905386929176921

In [49]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.90      0.92      2082
           1       0.88      0.91      0.89      1575

    accuracy                           0.91      3657
   macro avg       0.90      0.91      0.90      3657
weighted avg       0.91      0.91      0.91      3657

